In [1]:
# Generative graphs
import networkx as nx

from collections import defaultdict

from graphs import generate_graph, attach_graph_attributes
from measurements import get_graph_measurements, compare_graph_measurements, print_comparison_results
from recreate_graph import graph_to_training_dataframe, get_trained_model, recreate_by_priority_rank, preprocess_dataframe

Using TensorFlow backend.
/home/smoq/.virtualenvs/cna/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
graph_families = [
    {
        'name': 'erdos renyi',
        'func': nx.erdos_renyi_graph,
        'params': [
            {'n': 30, 'p': 0.05},
            {'n': 30, 'p': 0.1},
            {'n': 30, 'p': 0.3}
        ]
    },
    {
        'name': 'watts strogatz',
        'func': nx.watts_strogatz_graph,
        'params': [
            {'n': 30, 'k': 2, 'p': 0.05},
            {'n': 30, 'k': 2, 'p': 0.1},
            {'n': 30, 'k': 2, 'p': 0.3}
        ]
    },
    {
        'name': 'barabasi albert',
        'func': nx.barabasi_albert_graph,
        'params': [
            {'n': 30, 'm': 1},
            {'n': 30, 'm': 2},
            {'n': 30, 'm': 5}
        ]
    },
]

In [3]:
EPOCHS = 64
BATCH_SIZE = 64
VERBOSE = 0

In [4]:
results = defaultdict(list)
for graph_family in graph_families:
    for params in graph_family['params']:
        graph = generate_graph(graph_family['func'], params)
        attach_graph_attributes(graph)

        df = graph_to_training_dataframe(graph)
        df = preprocess_dataframe(df, graph.number_of_nodes())
        model = get_trained_model(df, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=VERBOSE)

        new_graph = recreate_by_priority_rank(graph, df, model)

        graph_measurements = get_graph_measurements(graph)
        new_graph_measurements = get_graph_measurements(new_graph)

        comparison = compare_graph_measurements(graph_measurements, new_graph_measurements)
        results[graph_family['name']].append(comparison)

900/900 [==============================] - 0s 24us/step
loss: 0.0030000330078635467, accuracy: 1.0
Cannot compute average_shortest_path_length - Graph is not connected.
Cannot compute diameter - Found infinite path length because the graph is not connected
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 25us/step
loss: 0.0030919077032012863, accuracy: 1.0
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 31us/step
loss: 0.003590305905478696, accuracy: 1.0
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


900/900 [==============================] - 0s 32us/step
loss: 0.002465346259996295, accuracy: 1.0
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 36us/step
loss: 0.0030346779989647785, accuracy: 1.0
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


900/900 [==============================] - 0s 46us/step
loss: 0.0036491044512432483, accuracy: 1.0
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 45us/step
loss: 0.0037036151904612778, accuracy: 1.0
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 52us/step
loss: 0.003823024006043043, accuracy: 1.0
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 56us/step
loss: 0.002641621758618082, accuracy: 1.0
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected


In [5]:
for key, value in results.items():
    print(key)
    for i, comparison_result in enumerate(value):
        print('graph #{}'.format(i+1))
        print_comparison_results(comparison_result)
        print()

erdos renyi
graph #1
(KS test p-value)  degree_centrality: 0.999999671760919               [passed: True]
(KS test p-value)  closeness_centrality: 5.5869816252578936e-08       [passed: False]
(KS test p-value)  betweenness_centrality: 0.10883839498780411        [passed: True]
(KS test p-value)  pagerank: 6.174024256235908e-05                    [passed: False]
(abs distance)     average_shortest_path_length: None                 [passed: None]
(abs distance)     diameter: None                                     [passed: None]
(abs distance)     degree_centralization: 0.3043478260869569          [passed: False]
(abs distance)     closeness_centralization: 0.820204505384201        [passed: False]
(abs distance)     betweenness_centralization: 0.014742698191933159   [passed: True]
(abs distance)     pagerank_centralization: 0.4734276354101936        [passed: False]
(abs distance)     density: 0.4827586206896552                        [passed: False]
(abs distance)     degree_assortativit